# Word2GM Training Data Pipeline

**Pipeline: Corpus file → TFRecord training artifacts (triplets and vocabulary)**

Use this notebook to prepare a Google 5gram corpora for Word2GM skip-gram training.

## Pipeline Workflow

1. **Input**: Preprocessed corpus file (e.g., `2019.txt`) in `/vast` NVMe storage
2. **Processing**: TensorFlow-native filtering, vocabulary building, and triplet generation
3. **Output**: TFRecord artifacts in organized subdirectories (e.g., `2019_artifacts/`)

### **Artifact Storage**
The pipeline creates year-specific subdirectories alongside the original text corpora:
<pre>
/vast/edk202/NLP_corpora/.../data/
├── 2018.txt
├── 2019.txt
├── 2020.txt
├── 2018_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
├── 2019_artifacts/
│   ├── triplets.tfrecord.gz
│   └── vocab.tfrecord.gz
└── 2020_artifacts/
    ├── triplets.tfrecord.gz
    └── vocab.tfrecord.gz
</pre>

## Set Up for Data Preparation

In [1]:
import sys
from pathlib import Path
import os

# Set project root and add src to path
PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
project_root = Path(PROJECT_ROOT)
src_path = project_root / 'src'

if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Enable autoreload for development
%load_ext autoreload
%autoreload 2

# Import TensorFlow quietly using our simplified silencing
from word2gm_fast.utils.tf_silence import import_tf_quietly
tf = import_tf_quietly(force_cpu=False)  # Allow GPU usage in notebook

# Basic imports
import numpy as np
import pandas as pd

# Import the batch processing function directly
from word2gm_fast.dataprep.simple_pipeline import run_pipeline

print("Setup complete - ready for data preparation")
print(f"✅ TensorFlow {tf.__version__} imported quietly")

2025-07-15 00:11:22.311087: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 00:11:22.327766: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752552682.346540 2977804 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752552682.352161 2977804 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752552682.366679 2977804 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Setup complete - ready for data preparation
✅ TensorFlow 2.19.0 imported quietly


## Print Resource Summary

In [2]:
# Import and run resource summary
from word2gm_fast.utils.resource_summary import print_resource_summary

print_resource_summary()

2025-07-15 00:11:27.876782: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


<pre>SYSTEM RESOURCE SUMMARY
=============================================
Hostname: cm011.hpc.nyu.edu

Job Allocation:
   CPUs: 14
   Memory: 125.0 GB
   Partition: short
   Job ID: 63738842
   Node list: cm011

Physical GPU Hardware:
   No physical GPUs allocated to this job

TensorFlow GPU Recognition:
   TensorFlow can access 0 GPU(s)
   Built with CUDA support: True
=============================================</pre>

## Prepare Corpora

Here, we run the data-preparation pipeline from start to finish — reading preprocessed ngram corpora, generating all valid triplets, extracting the vocabulary, and saving the triplets and vocabulary as `tfrecord` files.

### Options for Data Preparation

You can control which years are processed and how the batch preparation runs by adjusting the arguments to `batch_prepare_training_data`:

**Ways to specify years:**
- `year_range="2010"` — Process a single year (e.g., only 2010).
- `year_range="2010,2012,2015"` — Process a comma-separated list of years.
- `year_range="2010-2015"` — Process a range of years, inclusive (2010 through 2015).
- `year_range="2010,2012-2014,2016"` — Combine individual years and ranges (2010, 2012, 2013, 2014, 2016).

**Other options:**
- `compress` — If `True`, output TFRecords are gzip-compressed. If `False`, output is uncompressed.
- `show_progress` — If `True`, display a progress bar for each year.
- `show_summary` — If `True`, print a summary of the processed data for each year.
- `use_multiprocessing` — If `True`, process years in parallel using multiple CPU cores (recommended for large datasets).

**TensorFlow Logging:**
- TensorFlow logging is set to ERROR level to reduce verbose output
- The pipeline still works normally, but with cleaner console output
- Critical errors will still be displayed if they occur

See the function docstring or source for more advanced options.

In [ ]:
corpus_dir = '/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data'

results = run_pipeline(
    corpus_dir=corpus_dir,
    years="1600-1750",
    compress=False,
    show_progress=True,
    max_workers=None,
)


Processing 162 years: 1600-1800
Skipping 39 missing files
Using 24 parallel workers


2025-07-15 00:11:31.528978: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.547651: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.559913: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.608233: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.627596: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.634731: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at lookup_table_op.cc:1069 : INVALID_ARGUMENT: keys and values cannot be empty tensors.
2025-07-15 00:11:31.634752: I tensorflow/core/framewo

1644: FAILED (1/162)


2025-07-15 00:11:31.830826: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.832685: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.898333: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.906452: W tensorflow/core/framework/op_kernel.cc:1857] OP_REQUIRES failed at lookup_table_op.cc:1069 : INVALID_ARGUMENT: keys and values cannot be empty tensors.
2025-07-15 00:11:31.906476: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: INVALID_ARGUMENT: keys and values cannot be empty tensors.
2025-07-15 00:11:31.961171: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:11:31.9836

1615: FAILED (2/162)
1602: FAILED (3/162)


...

2025-07-15 00:12:29.629660: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:30.176433: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:30.176433: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1752: OK - 13,627 triplets (40.8s) [118/162]
1685: OK - 10,242 triplets (57.4s) [119/162]
1685: OK - 10,242 triplets (57.4s) [119/162]
1765: OK - 9,420 triplets (28.9s) [120/162]
1765: OK - 9,420 triplets (28.9s) [120/162]
1778: OK - 2,284 triplets (12.4s) [121/162]
1778: OK - 2,284 triplets (12.4s) [121/162]


2025-07-15 00:12:41.752769: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:42.209573: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:42.209573: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1766: OK - 8,810 triplets (34.0s) [122/162]


2025-07-15 00:12:42.625371: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:43.105133: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:12:43.105133: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [55]:
# SIMPLE PIPELINE: Clean, minimal wrapper
print("🚀 SIMPLE PIPELINE")
print("=" * 50)

# Import the clean, simple pipeline
%autoreload 2
from word2gm_fast.dataprep.simple_pipeline import run_pipeline

# Set corpus directory
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

print(f"Corpus directory: {corpus_dir}")
print()

# Example 1: Process a few test years
print("📋 TESTING: Small batch (3 years)")
results = run_pipeline(
    corpus_dir=corpus_dir,
    years="1683,1684,1685",
    compress=True,
    max_workers=2,
    show_progress=True
)

print()
print("📋 RESULTS:")
for year, result in results.items():
    if "error" in result:
        print(f"❌ {year}: {result['error']}")
    else:
        triplets = result['triplet_count']
        vocab = result['vocab_size']
        duration = result['duration']
        print(f"✅ {year}: {triplets:,} triplets, {vocab:,} vocab ({duration:.1f}s)")

print()
print("🎯 SIMPLE PIPELINE FEATURES:")
print("✅ Minimal complexity - just specify years and run")
print("✅ Automatic parallel processing")
print("✅ Clean progress output")
print("✅ Error handling per year")
print("✅ One-pass vocab-from-triplets approach")
print("✅ Zero UNK contamination guaranteed")

print()
print("📖 USAGE EXAMPLES:")
print('run_pipeline(corpus_dir, "1680-1690")        # Process decade')
print('run_pipeline(corpus_dir, "1684,1690,1695")   # Specific years')
print('run_pipeline(corpus_dir, "1680-1690", max_workers=4)  # Custom workers')

print()
print("=" * 50)
print("🎉 Simple pipeline ready for production!")

🚀 SIMPLE PIPELINE
Corpus directory: /vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data

📋 TESTING: Small batch (3 years)
Processing 3 years: 1683-1685
Using 2 parallel workers


2025-07-14 20:45:58.728821: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}


1683: OK - 43 triplets (0.6s) [1/3]


2025-07-14 20:45:59.584267: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}


1684: OK - 2,959 triplets (4.6s) [2/3]
1685: OK - 10,242 triplets (9.5s) [3/3]

Completed in 10.2s
✅ Successful: 3 years
📊 Total triplets: 13,244
📚 Average vocab: 2,526

📋 RESULTS:
✅ 1683: 43 triplets, 104 vocab (0.6s)
✅ 1684: 2,959 triplets, 2,533 vocab (4.6s)
✅ 1685: 10,242 triplets, 4,942 vocab (9.5s)

🎯 SIMPLE PIPELINE FEATURES:
✅ Minimal complexity - just specify years and run
✅ Automatic parallel processing
✅ Clean progress output
✅ Error handling per year
✅ One-pass vocab-from-triplets approach
✅ Zero UNK contamination guaranteed

📖 USAGE EXAMPLES:
run_pipeline(corpus_dir, "1680-1690")        # Process decade
run_pipeline(corpus_dir, "1684,1690,1695")   # Specific years
run_pipeline(corpus_dir, "1680-1690", max_workers=4)  # Custom workers

🎉 Simple pipeline ready for production!
1685: OK - 10,242 triplets (9.5s) [3/3]

Completed in 10.2s
✅ Successful: 3 years
📊 Total triplets: 13,244
📚 Average vocab: 2,526

📋 RESULTS:
✅ 1683: 43 triplets, 104 vocab (0.6s)
✅ 1684: 2,959 triplets

In [57]:
results = run_pipeline(
    corpus_dir=corpus_dir,
    years="1680-1689",
    compress=True,
    max_workers=4,  # Use more workers for larger batches
    show_progress=True
)


Processing 10 years: 1680-1689
Using 4 parallel workers


2025-07-14 20:47:14.682083: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}
2025-07-14 20:47:14.684088: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),m

1683: OK - 43 triplets (0.7s) [1/10]
1682: OK - 53 triplets (0.7s) [2/10]
1680: OK - 51 triplets (0.7s) [3/10]


2025-07-14 20:47:15.146576: E tensorflow/core/framework/node_def_util.cc:680] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_4}}


1686: OK - 4 triplets (0.4s) [4/10]
1687: OK - 36 triplets (0.6s) [5/10]
1687: OK - 36 triplets (0.6s) [5/10]
1688: OK - 397 triplets (1.4s) [6/10]
1688: OK - 397 triplets (1.4s) [6/10]
1681: OK - 2,095 triplets (3.4s) [7/10]
1681: OK - 2,095 triplets (3.4s) [7/10]
1689: OK - 21 triplets (0.5s) [8/10]
1689: OK - 21 triplets (0.5s) [8/10]
1684: OK - 2,959 triplets (4.7s) [9/10]
1684: OK - 2,959 triplets (4.7s) [9/10]
1685: OK - 10,242 triplets (9.7s) [10/10]

Completed in 10.5s
✅ Successful: 10 years
📊 Total triplets: 15,901
📚 Average vocab: 1,077
1685: OK - 10,242 triplets (9.7s) [10/10]

Completed in 10.5s
✅ Successful: 10 years
📊 Total triplets: 15,901
📚 Average vocab: 1,077


In [4]:
# TEST: Improved Multiprocessing Silencing
print("🔇 TESTING IMPROVED MULTIPROCESSING SILENCING")
print("=" * 60)

# Ensure module path is available
import sys
import os
from pathlib import Path
PROJECT_ROOT = '/scratch/edk202/word2gm-fast'
src_path = Path(PROJECT_ROOT) / 'src'
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

# Import the pipeline function
from word2gm_fast.dataprep.simple_pipeline import run_pipeline

# Set corpus directory
corpus_dir = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng-fiction/5gram_files/6corpus/yearly_files/data"

# Set up TensorFlow environment for worker processes
from word2gm_fast.utils.tf_silence import setup_tf_environment
setup_tf_environment(force_cpu=True)

print("✅ TensorFlow environment configured for workers")
print(f"TF_CPP_MIN_LOG_LEVEL: {os.environ.get('TF_CPP_MIN_LOG_LEVEL', 'NOT SET')}")
print(f"TF_ENABLE_ONEDNN_OPTS: {os.environ.get('TF_ENABLE_ONEDNN_OPTS', 'NOT SET')}")

# Test with a small batch
print()
print("📋 Testing with small batch (should be much quieter)...")
results = run_pipeline(
    corpus_dir=corpus_dir,
    years="1683,1684",  # Just 2 files for testing
    compress=True,
    max_workers=2,
    show_progress=True
)

print()
print("📋 RESULTS:")
for year, result in results.items():
    if "error" in result:
        print(f"❌ {year}: {result['error']}")
    else:
        triplets = result['triplet_count']
        vocab = result['vocab_size']
        duration = result['duration']
        print(f"✅ {year}: {triplets:,} triplets, {vocab:,} vocab ({duration:.1f}s)")

print()
print("🎯 If you still see TensorFlow messages above, they are:")
print("• Coming from TensorFlow's C++ core (very hard to suppress)")
print("• Normal INFO messages, not errors")
print("• Much reduced compared to before!")

🔇 TESTING IMPROVED MULTIPROCESSING SILENCING
✅ TensorFlow environment configured for workers
TF_CPP_MIN_LOG_LEVEL: 3
TF_ENABLE_ONEDNN_OPTS: 0

📋 Testing with small batch (should be much quieter)...
Processing 2 years: 1683-1684
Using 2 parallel workers


2025-07-15 00:06:16.018190: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-07-15 00:06:16.018190: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-07-15 00:06:16.731085: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:06:16.731085: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:06:17.177532: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:06:17.177532: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1683: OK - 43 triplets (1.2s) [1/2]


2025-07-15 00:06:17.686904: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:06:19.418593: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-07-15 00:06:19.418593: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1684: OK - 2,959 triplets (3.9s) [2/2]

Completed in 4.0s
✅ Successful: 2 years
📊 Total triplets: 3,002
📚 Average vocab: 1,318

📋 RESULTS:
✅ 1683: 43 triplets, 104 vocab (1.2s)
✅ 1684: 2,959 triplets, 2,533 vocab (3.9s)

🎯 If you still see TensorFlow messages above, they are:
• Coming from TensorFlow's C++ core (very hard to suppress)
• Normal INFO messages, not errors
• Much reduced compared to before!


In [ ]:
# TEST: Updated TensorFlow Silencing (No INFO/WARNING)
print("🔕 TESTING: TF_CPP_MIN_LOG_LEVEL=2 (No INFO/WARNING)")
print("=" * 60)

# Reset environment and apply updated silencing
import os
os.environ.pop('TF_CPP_MIN_LOG_LEVEL', None)  # Clear old setting

# Apply new silencing settings
from word2gm_fast.utils.tf_silence import setup_tf_environment
setup_tf_environment(force_cpu=True)

print("✅ Updated TensorFlow environment:")
print(f"TF_CPP_MIN_LOG_LEVEL: {os.environ.get('TF_CPP_MIN_LOG_LEVEL', 'NOT SET')} (should suppress INFO and WARNING)")
print(f"TF_SUPPRESS_LOGS: {os.environ.get('TF_SUPPRESS_LOGS', 'NOT SET')}")

# Test with a single year
print()
print("📋 Testing single year (should be very quiet)...")
results = run_pipeline(
    corpus_dir=corpus_dir,
    years="1683",  # Single small file
    compress=True,
    max_workers=1,  # Single worker for cleaner output
    show_progress=True
)

print()
print("📋 RESULT:")
for year, result in results.items():
    if "error" in result:
        print(f"❌ {year}: {result['error']}")
    else:
        triplets = result['triplet_count']
        vocab = result['vocab_size']
        duration = result['duration']
        print(f"✅ {year}: {triplets:,} triplets, {vocab:,} vocab ({duration:.1f}s)")

print()
print("🎯 TF_CPP_MIN_LOG_LEVEL=2 should suppress:")
print("• INFO messages (I) - Local rendezvous, End of sequence, etc.")
print("• WARNING messages (W) - OP_REQUIRES failed, etc.")
print("• Keep only ERROR and FATAL messages")